In [ ]:
# todo:
# - understand how the training data text is represented for chain of thought ot 3SUM sequences
# - function for evaluating accuracy
# - understand decoder implementation
# - function to print rank-n for layer m and then iterate over using this


# experiments:
# - write the claims of the paper: COT is uninterpretable as it can be hidden, but this hidden COT still increases performs
# - see where it is being replaced with '.' and where it is just the numbers
# - check this across layer, across rank
# - is this gradual or not gradual?
# - check if these decoded sequences from different ranks and layers are in anyway simialr to the actual trianing dataset.
# - write the punchline: you only see the decoded text of COT, but not what is there in the rest of the logits as information.

In [1]:
from src.match3 import *
from transformers import AutoModelForCausalLM, AutoTokenizer, AutoConfig
from src.utils import InputEmbedCausalTransformer
import json
import numpy as np
import os 
import pandas as pd
import torch

/home/loki/anaconda3/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
MODEL_PATH = "/home/loki/projects/filler_tokens_repo/output_dir/2024-08-14-22-matchdata-checkpoint-final/model_weights.pt"
CONFIG_FILE = "/home/loki/projects/filler_tokens_repo/misc/llama_d384l4h6.json"

In [4]:
train_df = pd.read_csv('data/matchdata_trainset_2024-08-13.csv', header=None, names=["text"])
test_df = pd.read_csv('data/matchdata_testset_2024-08-13.csv', header=None, names=["text"])

In [5]:
train_df.iloc[1].text

' 001 991 866 226 697 019 100 711 522 541 P 1- 9 2- 8 0- 7 4- 6 0- 1 6- 1 0- 2 8- 5 0- 2 1- 7 1- 1 1- 8- 5- 6- 6- 0 7- 2 1- 4 9- 3 2- 2 4- 5 2- 7 6- 6 2- 7 2- 8 2- 0 3- 1 5- 2 3- 6 3- 3 3- 7 9- 7 4- 6 6- 7 7- 8 8- 9 4- 3 6- 1 7- 7 5- 5 9- 5 7- 8 6- 2 6- 1 7- 3 9- 2 9- 3 9- 0 A True'

In [6]:
test_df.iloc[1].text

' 204 161 783 380 758 113 046 162 310 217 P . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . A False'

In [7]:
test_set = Match3VectorDataset(test_df, 3, 10, 10, 'P')
print(test_set.input_dim)

validate encodings
raw input 0  433 450 421 129 107 924 489 711 540 034 P 0- 3 0- 4 3- 2 4- 0 5- 5 0- 8 7- 1 0- 7 9- 4 1- 8 3- 5 1- 7 1- 3 6- 8 1- 1 8- 9 9- 4 3- 5 4- 2 5- 5 6- 8 2- 2 2- 6 9- 5 3- 2 5- 0 6- 5 3- 0 8- 9 9- 5 4- 2 6- 8 7- 1 8- 4 4- 3 6- 3 5- 6 8- 4 9- 8 7- 9 8- 9 6- 1 7- 2 7- 7 9- 7 9- 5 A False
encoded sample 0 {'input_ids': tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [1., 0., 0.,  ..., 0., 0., 0.],
        [1., 0., 0.,  ..., 0., 0., 0.],
        [1., 0., 0.,  ..., 0., 0., 0.]], dtype=torch.float16), 'labels': tensor([-100, -100, -100, -100, -100, -100, -100, -100, -100, -100,    5,   28,
           5,   29,    8,   27,    9,   25,   10,   30,    5,   33,   12,   26,
           5,   32,   14,   29,    6,   33,    8,   30,    6,   32,    6,   28,
          11,   33,    6,   26,   13,   34,   14,   29,    8,   30,    9,   27,
          10,   30,   11,   33,    7,   27,    7,  

In [8]:
test_set[0]

{'input_ids': tensor([[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [1., 0., 0.,  ..., 0., 0., 0.],
         [1., 0., 0.,  ..., 0., 0., 0.],
         [1., 0., 0.,  ..., 0., 0., 0.]], dtype=torch.float16),
 'labels': tensor([-100, -100, -100, -100, -100, -100, -100, -100, -100, -100,    5,   28,
            5,   29,    8,   27,    9,   25,   10,   30,    5,   33,   12,   26,
            5,   32,   14,   29,    6,   33,    8,   30,    6,   32,    6,   28,
           11,   33,    6,   26,   13,   34,   14,   29,    8,   30,    9,   27,
           10,   30,   11,   33,    7,   27,    7,   31,   14,   30,    8,   27,
           10,   25,   11,   30,    8,   25,   13,   34,   14,   30,    9,   27,
           11,   33,   12,   26,   13,   29,    9,   28,   11,   28,   10,   31,
           13,   29,   14,   33,   12,   34,   13,   34,   11,   26,   12,   27,
           12,   32,   14,   32,   14,   30,  

In [9]:
test_set.word_index_map

{'True': 1, 'False': 2, '.': 3, 'A': 4, 'P': 5}

In [10]:
class BlockOutputWrapper(torch.nn.Module):
    def __init__(self, block, unembed_matrix, norm):
        super().__init__()
        self.block = block
        self.unembed_matrix = unembed_matrix
        self.norm = norm
        self.block_output_unembedded = None

    def forward(self, *args, **kwargs):
        output = self.block(*args, **kwargs)
        if isinstance(output, tuple):
            self.block_output_unembedded = self.unembed_matrix(self.norm(output[0]))
            return output
        else:
            self.block_output_unembedded = self.unembed_matrix(self.norm(output))
            return output

    def reset_block_output(self):
        self.block_output_unembedded = None

class LlamaHelper:
    def __init__(self, config_file, model_path, test_set):
        self.device = "cuda" if torch.cuda.is_available() else "cpu"
        config = AutoConfig.from_pretrained(config_file)
        model = InputEmbedCausalTransformer(AutoModelForCausalLM.from_config(config), test_set.input_dim)
        state_dict = torch.load(model_path)
        model.load_state_dict(state_dict, strict=False)
        model = model.to(self.device)
        self.model = model
        self.word_index_map = test_set.word_index_map
        self.data_len = test_set.data_len
        self.mod = test_set.mod
        
        for i, layer in enumerate(self.model.base_model.model.layers):
            self.model.base_model.model.layers[i] = BlockOutputWrapper(layer, self.model.base_model.lm_head, self.model.base_model.model.norm)

    def decode_tensors(self, sequence):
        decoded_sequence = []
        marker_found = False

        for token in sequence:
            token = token.item()
            if token == -100:
                if not marker_found:
                    decoded_sequence.append("[MASK]")
                continue
            elif token == 0:
                decoded_sequence.append("[EOS]")
                break  # Stop decoding after EOS
            elif token < len(self.word_index_map):
                # Regular word
                word = list(self.word_index_map.keys())[list(self.word_index_map.values()).index(token)]
                decoded_sequence.append(word)
                if word in ["A", "P"]:
                    marker_found = True
            else:
                # Handle digit labels
                offset = len(self.word_index_map)
                if token < offset + self.data_len * 2:
                    # Tuple index encoding
                    idx = (token - offset) % self.data_len
                    tuple_pos = (token - offset) // self.data_len
                    decoded_sequence.append(f"{tuple_pos}-{idx}")
                else:
                    # Single digit or digit in tuple
                    char_pos = (token - offset - self.data_len * 2) // self.mod
                    digit = (token - offset - self.data_len * 2) % self.mod
                    if char_pos == 0 or len(decoded_sequence) == 0 or not decoded_sequence[-1][-1].isdigit():
                        decoded_sequence.append(str(digit))
                    else:
                        decoded_sequence[-1] += str(digit)

        return " ".join(decoded_sequence)


    def get_logits(self, inputs):
        with torch.no_grad():
            input_ids = inputs['input_ids'].float().to(self.device).unsqueeze(0)
            out = self.model(input_ids)
            return out.logits

    def set_add_attn_output(self, layer, add_output):
        self.model.base_model.model.layers[layer].attn_add_tensor(add_output)

    def get_attn_activations(self, layer):
        return self.model.base_model.model.layers[layer].get_attn_activations()

    def reset_all_layers(self):
        for layer in self.model.base_model.model.layers:
            layer.reset_block_output()

    def print_logit_progression(self, inputs, layer_number=None, rank=1, sequence_length=None):
        self.reset_all_layers()
        logits = self.get_logits(inputs)
        num_layers = len(self.model.base_model.model.layers)
        seq_length = inputs['input_ids'].size(1) if sequence_length is None else min(sequence_length, inputs['input_ids'].size(1))
        result_dict = {}

        def process_layer(layer_idx):
            tokens = []
            values = []
            layer = self.model.base_model.model.layers[layer_idx] if layer_idx < num_layers else None
            for j in range(seq_length):
                if layer and layer.block_output_unembedded is not None:
                    layer_logits = layer.block_output_unembedded[0][j]
                elif layer_idx == num_layers:  # Final output
                    layer_logits = logits[0][j]
                else:
                    continue
                val, idx = torch.topk(layer_logits, rank)
                tokens.append(self.decode_tensors(idx[-1].unsqueeze(-1)).strip())
                values.append(val[-1].item())
            return tokens, values

        if layer_number is not None:
            if layer_number > num_layers:
                print(f"Error: Layer number {layer_number} is out of range. Max layer is {num_layers}.")
                return {}
            tokens, values = process_layer(layer_number)
            layer_name = f"h{layer_number}_out" if layer_number < num_layers else "h_out"
            print(f"{rank}th highest logit for {layer_name}:")
            print("".join(tokens))
            print(" ".join([f"{v:.2f}" for v in values]))
            result_dict[layer_name] = "".join(tokens)
        else:
            print(f"{rank}th highest logit:")
            for i in range(num_layers + 1):
                tokens, values = process_layer(i)
                layer_name = f"h{i}_out" if i < num_layers else "h_out"
                print(f"{layer_name:<5}: " + "".join(tokens))
                result_dict[layer_name] = "".join(tokens)

        return result_dict

In [11]:
model = LlamaHelper(CONFIG_FILE, MODEL_PATH, test_set)

In [12]:
train_df.iloc[60].text

' 920 570 551 556 703 290 134 989 839 671 P . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . A False'

In [13]:
test_df.iloc[60].text

' 459 684 159 082 550 700 089 439 835 429 P . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . A False'

In [14]:
model.print_logit_progression(test_set[50], rank=1, sequence_length=50)
print("\n")
model.print_logit_progression(test_set[50], rank=2, sequence_length=50)
print("_"*114)

We detected that you are passing `past_key_values` as a tuple and this is deprecated and will be removed in v4.43. Please use an appropriate `Cache` class (https://huggingface.co/docs/transformers/v4.41.3/en/internal/generation_utils#transformers.Cache)


1th highest logit:
h0_out: [EOS]A[EOS][EOS]0-25[EOS]10-0.0-1[EOS]AA162000A590-04763000-20-20-0.1103339....8A[EOS][EOS][EOS]
h1_out: 3[EOS][EOS]5660[EOS]0-02.2428899AA88AAAA885A25888688869992211166
h2_out: 0-20-20-2.0-10-5.0-70-00-0.AAA....A...........A.........9.......[EOS].
h3_out: 0-50-2...0-0.0-00-00-0........................................
h_out: 0-50-2...0-0.0-00-00-0........................................


2th highest logit:
h0_out: 3[EOS]A40-60-88770-1.351201241516.5536330-70-70-70-025244410-00-00-50-521A99
h1_out: [EOS]1A153.30-480-16246633..96[EOS]6..9645A066686668766A588855
h2_out: 00-6.0-1.0-00-10-00-70-10-1[EOS]2288808[EOS]True8AATrueTrue8TrueTrueTrue[EOS]TrueA8[EOS]True888True799ATrueTrueTrueTrueA6
h3_out: .0-50-10-10-1.0-00-7.0-10-0TrueTrueTrueFalseFalseTrueTrueTrueTrueTrueTrueTrueTrueTrueTrueFalseFalseTrueTrueTrueFalseFalseFalseFalseFalseFalseFalseFalseTrueFalseFalseFalseTrueFalseFalseFalseFalseFalseFalse
h_out: .0-50-10-10-1.0-00-7.0-10-0TrueTrueTrueFalseFalseTrueTru

In [15]:
r1 = model.print_logit_progression(test_set[50], rank=1, sequence_length=50)
r2 = model.print_logit_progression(test_set[50], rank=2, sequence_length=50)

1th highest logit:
h0_out: [EOS]A[EOS][EOS]0-25[EOS]10-0.0-1[EOS]AA162000A590-04763000-20-20-0.1103339....8A[EOS][EOS][EOS]
h1_out: 3[EOS][EOS]5660[EOS]0-02.2428899AA88AAAA885A25888688869992211166
h2_out: 0-20-20-2.0-10-5.0-70-00-0.AAA....A...........A.........9.......[EOS].
h3_out: 0-50-2...0-0.0-00-00-0........................................
h_out: 0-50-2...0-0.0-00-00-0........................................
2th highest logit:
h0_out: 3[EOS]A40-60-88770-1.351201241516.5536330-70-70-70-025244410-00-00-50-521A99
h1_out: [EOS]1A153.30-480-16246633..96[EOS]6..9645A066686668766A588855
h2_out: 00-6.0-1.0-00-10-00-70-10-1[EOS]2288808[EOS]True8AATrueTrue8TrueTrueTrue[EOS]TrueA8[EOS]True888True799ATrueTrueTrueTrueA6
h3_out: .0-50-10-10-1.0-00-7.0-10-0TrueTrueTrueFalseFalseTrueTrueTrueTrueTrueTrueTrueTrueTrueTrueFalseFalseTrueTrueTrueFalseFalseFalseFalseFalseFalseFalseFalseTrueFalseFalseFalseTrueFalseFalseFalseFalseFalseFalse
h_out: .0-50-10-10-1.0-00-7.0-10-0TrueTrueTrueFalseFalseTrueTrueT

In [18]:
# from tqdm import tqdm
# import pandas as pd

In [19]:
# rank_decoded = []

# for idx in tqdm(range(1000)):
#     r1 = model.print_logit_progression(test_set[idx], rank=1, sequence_length=50)
#     r2 = model.print_logit_progression(test_set[idx], rank=2, sequence_length=50)
#     rank_decoded.append({"r1":r1["h_out"], "r2":r2["h_out"]})